In [1]:
import pandas as pd

In [3]:
df = pd.read_parquet('c:/fomo_data/comments_added.parquet')

In [4]:
events = pd.read_csv('c:/fomo_data/events.csv')

In [5]:
events.head()

,Event Type,Organization Name,Facility Name,Direction,City,County,State,Create Time,Close Time,Event Description,Responding Organization Id,Latitude,Longitude
0,football game,Meadowlands Complex,MetLife Stadium,NaN,at East Rutherford,Bergen,NJ,12/13/2025 03:00:00 PM,NaN,NaN,Meadowlands Complex,40.813460,-74.074287
1,concert,Meadowlands Complex,MetLife Stadium,NaN,at East Rutherford,Bergen,NJ,08/04/2023 18:00,NaN,NaN,Meadowlands Complex,40.813460,-74.074287
2,concert,Meadowlands Complex,MetLife Stadium,NaN,at East Rutherford,Bergen,NJ,07/29/2023 07:00:00 PM,NaN,NaN,Meadowlands Complex,40.813460,-74.074287
3,concert,Meadowlands Complex,MetLife Stadium,NaN,at East Rutherford,Bergen,NJ,07/06/2023 19:30,NaN,NaN,Meadowlands Complex,40.813460,-74.074287
4,fair,Meadowlands Complex,Meadowlands Sports Complex,NaN,East Rutherford,Bergen,NJ,06/22/2023 12:00:00 PM,NaN,NaN,Meadowlands Complex,40.814458,-74.071969


In [7]:
events.drop(columns=['Organization Name', 'Direction', 'City', 'County', 'State', 'Close Time', 'Event Description', 'Responding Organization Id'], inplace=True)

In [8]:
events.head()

,Event Type,Facility Name,Create Time,Latitude,Longitude
0,football game,MetLife Stadium,12/13/2025 03:00:00 PM,40.813460,-74.074287
1,concert,MetLife Stadium,08/04/2023 18:00,40.813460,-74.074287
2,concert,MetLife Stadium,07/29/2023 07:00:00 PM,40.813460,-74.074287
3,concert,MetLife Stadium,07/06/2023 19:30,40.813460,-74.074287
4,fair,Meadowlands Sports Complex,06/22/2023 12:00:00 PM,40.814458,-74.071969


In [9]:
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13377 entries, 0 to 13376
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Event Type     13377 non-null  object 
 1   Facility Name  13377 non-null  object 
 2   Create Time    13377 non-null  object 
 3   Latitude       13377 non-null  float64
 4   Longitude      13377 non-null  float64
dtypes: float64(2), object(3)
memory usage: 522.7+ KB


In [11]:
events['Event Type'] = events['Event Type'].astype('category')
events['Facility Name'] = events['Facility Name'].astype('category')